In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import pandas as pd

In [6]:
u_min, u_max = -0.21, 0.0
x_min, x_max = 0.0, 1.0
y_min, y_max = 0.0, 1.0
t_f = 10
f_min, f_max = -3.0, 0.0

def hard_constraint(x_in, U_theta):
    delta_u = u_max - u_min
    delta_x = x_max - x_min
    delta_y = y_max - y_min

    X = x_in[0]
    Y = x_in[1]
    tau = x_in[-1]
    x = X*delta_x + x_min
    y = Y*delta_y + y_min
    t = tau * t_f
    u_theta = U_theta*delta_u + u_min

    # se siamo sui bordi (qualsiasi dimensione) u=0, altrimenti abbiamo applicato una trasformazione non lineare a u_theta
    u = u_theta * (x-x_min) *(x-x_max) * (y-y_min) * (y-y_max) * t
    U = (u - u_min)/delta_u # forma esplicita: riga 72 in 73, poi righe 66-69 

    # output normalizzato
    return U

In [ ]:
import torch
import pandas as pd
import numpy as np

# Paths and setup
model_path = '/Users/francesco/Desktop/PAI1/training/KAN_2/model/model_324.pt'
csv_path = 'matlab/only_damp/space_time_points.csv'
output_path = 'matlab/only_damp'
batch_size = 500

# Load model and data
model = torch.load(model_path)
model.eval()

df = pd.read_csv(csv_path)

# Convert pandas Series to numpy arrays
X = df['x'].to_numpy()
Y = df['y'].to_numpy()
T = df['t'].to_numpy()

n_samples = len(X)
predictions = np.zeros(n_samples)

# Prepare input data as tensor
inputs = np.column_stack((X, Y, T))
inputs = torch.FloatTensor(inputs)

# Process in batches
with torch.no_grad():
    for i in range(0, n_samples, batch_size):
        end_idx = min(i + batch_size, n_samples)  # Ensure we don't go past the array end
        batch_inputs = inputs[i:end_idx]
        batch_outputs = model(batch_inputs)
        predictions[i:end_idx] = batch_outputs.cpu().numpy().flatten()[:end_idx-i]  # Only take the needed values

# Save predictions
df['predicted_value'] = predictions
df.to_csv(output_path + '/predictions.csv', index=False)  # Added '/predictions.csv' to the path

print(f"Predictions completed and saved to {output_path}")
print("\nSample predictions:")
print(df.head())

# Print statistics about the predictions
print("\nPrediction statistics:")
print(f"Mean: {predictions.mean():.4f}")
print(f"Std: {predictions.std():.4f}")
print(f"Min: {predictions.min():.4f}")
print(f"Max: {predictions.max():.4f}")


Predictions completed and saved to matlab/only_damp

Sample predictions:
      x    y    t  predicted_value
0  0.00  1.0  0.0              1.0
1  1.00  1.0  0.0              1.0
2  1.00  0.0  0.0              1.0
3  0.00  0.0  0.0              1.0
4  0.05  0.0  0.0              1.0

Prediction statistics:
Mean: 51365.3620
Std: 171145.4632
Min: -227.1750
Max: 764751.5625


/var/folders/kk/2c_8134s1jqf2s0cx82ytg3h0000gn/T/ipykernel_53029/3601177203.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path)
